# **Apache Spark**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
import time

In [ ]:
schema = StructType([
    StructField("video_id", StringType(), True),
    StructField("uploader", StringType(), True),
    StructField("age", FloatType(), True),
    StructField("category", StringType(), True),
    StructField("length", FloatType(), True),
    StructField("views", FloatType(), True),
    StructField("rating", FloatType(), True),
    StructField("ratings_count", FloatType(), True),
    StructField("comments_count", FloatType(), True),
    StructField("related_videos", StringType(), True) 
])

In [ ]:
spark = SparkSession.builder \
    .appName("YouTube Analysis") \
    .getOrCreate()

In [ ]:
df = spark.read.csv("hdfs://localhost:9000/hdfs/path/Hive/merged_youtube_data", 
                     header=False,  
                     schema=schema, 
                     sep="\t")

In [ ]:
df.printSchema()

### **Videos with High Views but Low Ratings**

In [ ]:
high_views_threshold = 10000 
low_ratings_threshold = 3.0
high_views_low_ratings_df = df.filter((df.views > high_views_threshold) & (df.rating < low_ratings_threshold))

In [ ]:
start_time = time.time()
high_views_low_ratings_df = high_views_low_ratings_df.select("video_id", "uploader", "views", "rating")
end_time = time.time()

In [ ]:
high_views_low_ratings_df.show(truncate=False)

In [ ]:
execution_time = end_time - start_time
print(f"Execution time for the operation: {execution_time} seconds")

In [ ]:
high_views_low_ratings_df.write.csv("hdfs://localhost:9000/hdfs/path/Spark/high_views_low_ratings", 
                                     header=True, 
                                     mode="overwrite")

### **Average Video Length by Category**

In [ ]:
start_time = time.time()
average_length_df = df.groupBy("category").agg({"length": "avg"}).withColumnRenamed("avg(length)", "average_length")
end_time = time.time()

In [ ]:
average_length_df.show(truncate=False)

In [ ]:
execution_time = end_time - start_time
print(f"Execution time for the operation: {execution_time} seconds")

In [ ]:
average_length_df.write.csv("hdfs://localhost:9000/hdfs/path/Spark/avg_video_length_by_category", 
                             header=True, 
                             mode="overwrite") 

### **Average Rating by Category**

In [ ]:
start_time = time.time()
average_rating_df = df.groupBy("category").agg({"rating": "avg"}) \
    .withColumnRenamed("avg(rating)", "average_rating")
end_time = time.time()

In [ ]:
average_rating_df.show(truncate=False)

In [ ]:
execution_time = end_time - start_time
print(f"Execution time for the operation: {execution_time} seconds")

In [ ]:
average_rating_df.write.csv("hdfs://localhost:9000/hdfs/path/Spark/avg_rating_by_category", 
                             header=True, 
                             mode="overwrite")

### **Average Views by Category**

In [ ]:
start_time = time.time()
average_views_df = df.groupBy("category").agg({"views": "avg"}) \
    .withColumnRenamed("avg(views)", "average_views")
end_time = time.time()

In [ ]:
average_views_df.show(truncate=False)

In [ ]:
execution_time = end_time - start_time
print(f"Execution time for the operation: {execution_time} seconds")

In [ ]:
average_views_df.write.csv("hdfs://localhost:9000/hdfs/path/Spark/avg_views_by_category", 
                            header=True, 
                            mode="overwrite")

### **Total Comments by Category**

In [ ]:
start_time = time.time()
total_comments_df = df.groupBy("category").agg({"comments_count": "sum"}) \
    .withColumnRenamed("sum(comments_count)", "total_comments")
end_time = time.time()

In [ ]:
total_comments_df.show(truncate=False)

In [ ]:
execution_time = end_time - start_time
print(f"Execution time for the operation: {execution_time} seconds")

In [ ]:
total_comments_df.write.csv("hdfs://localhost:9000/yt/Spark/total_comments_by_category", 
                             header=True, 
                             mode="overwrite")

### **Total Videos by Uploader**

In [ ]:
start_time = time.time()
total_videos_by_uploader_df = df.groupBy("uploader").agg({"video_id": "count"}) \
    .withColumnRenamed("count(video_id)", "total_videos")
end_time = time.time()

In [ ]:
total_videos_by_uploader_df.show(truncate=False)

In [ ]:
execution_time = end_time - start_time
print(f"Execution time for the operation: {execution_time} seconds")

In [ ]:
total_videos_by_uploader_df.write.csv("hdfs://localhost:9000/hdfs/path/Spark/total_videos_by_uploader", 
                                        header=True, 
                                        mode="overwrite")

### **Top 10 Most Viewed Videos**

In [ ]:
start_time = time.time()
top_10_most_viewed_df = df.orderBy(df.views.desc()).limit(10)
end_time = time.time()

In [ ]:
top_10_most_viewed_df.show(truncate=False)

In [ ]:
execution_time = end_time - start_time
print(f"Execution time for the operation: {execution_time} seconds")

In [ ]:
top_10_most_viewed_df.write.csv("hdfs://localhost:9000/hdfs/path/Spark/top_10_most_viewed_videos", 
                                  header=True, 
                                  mode="overwrite")

### **Top 5 Longest Videos**

In [ ]:
start_time = time.time()
top_5_longest_videos_df = df.orderBy(df.length.desc()).limit(5)
end_time = time.time()

In [ ]:
top_5_longest_videos_df.show(truncate=False)

In [ ]:
execution_time = end_time - start_time
print(f"Execution time for the operation: {execution_time} seconds")

In [ ]:
top_5_longest_videos_df.write.csv("hdfs://localhost:9000/hdfs/path/Spark/top_5_longest_videos", 
                                    header=True, 
                                    mode="overwrite")

### **Top 5 Uploaders by Number of Videos**

In [ ]:
start_time = time.time()
top_5_uploaders_df = df.groupBy("uploader").agg({"video_id": "count"}) \
    .withColumnRenamed("count(video_id)", "total_videos") \
    .orderBy("total_videos", ascending=False) \
    .limit(5)
end_time = time.time()

In [ ]:
top_5_uploaders_df.show(truncate=False)

In [ ]:
execution_time = end_time - start_time
print(f"Execution time for the operation: {execution_time} seconds")

In [ ]:
top_5_uploaders_df.write.csv("hdfs://localhost:9000/hdfs/path/Spark/top_5_uploaders_by_number_of_videos", 
                              header=True, 
                              mode="overwrite")

### **Length vs Views**

In [ ]:
start_time = time.time()
length_vs_views_df = df.groupBy("length").agg({"views": "avg"}) \
    .withColumnRenamed("avg(views)", "avg_views")
end_time = time.time()

In [ ]:
length_vs_views_df.show(truncate=False)

In [ ]:
execution_time = end_time - start_time
print(f"Execution time for the operation: {execution_time} seconds")

In [ ]:
length_vs_views_df.write.csv("hdfs://localhost:9000/hdfs/path/Spark/length_vs_views", 
                              header=True, 
                              mode="overwrite")

### **Videos with Ratings Above 4.5**

In [ ]:
start_time = time.time()
high_rating_df = df.filter(df.rating > 4.5)
end_time = time.time()

In [ ]:
high_rating_df.show(truncate=False)

In [ ]:
execution_time = end_time - start_time
print(f"Execution time for the operation: {execution_time} seconds")

In [ ]:
high_rating_df.write.csv("hdfs://localhost:9000/hdfs/path/Spark/videos_with_ratings_above_4_5", 
                          header=True, 
                          mode="overwrite")

In [ ]:
spark.stop()

# **Hive**

#### Run the following commands on Hive

### **Top 10 Most Viewed Videos**

In [ ]:
INSERT OVERWRITE DIRECTORY '/hdfs/path/Hive/top_10_most_viewed_videos'
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY '\t' 
SELECT video_id, views
FROM merged_youtube_data
ORDER BY views DESC
LIMIT 10;

### **Average Rating by Category**

In [ ]:
INSERT OVERWRITE DIRECTORY '/hdfs/path/Hive/avg_rating_by_category'
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
SELECT category, AVG(rating) AS avg_rating
FROM merged_youtube_data
GROUP BY category;

### **Videos with Ratings Greater Than 4.5**

In [ ]:
INSERT OVERWRITE DIRECTORY '/hdfs/path/Hive/videos_with_ratings_above_4_5'
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
SELECT video_id, rating
FROM merged_youtube_data
WHERE rating > 4.5;

### **Average Views per Category**

In [ ]:
INSERT OVERWRITE DIRECTORY '/hdfs/path/Hive/avg_views_by_category'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
SELECT category, COUNT(*) AS total_videos, AVG(rating) AS avg_rating
FROM merged_youtube_data
GROUP BY category;

### **Top 5 Uploaders by Number of Videos**

In [ ]:
INSERT OVERWRITE DIRECTORY '/hdfs/path/Hive/top_5_uploaders_by_number_of_videos'
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
SELECT uploader, COUNT(video_id) AS video_count
FROM merged_youtube_data
GROUP BY uploader
ORDER BY video_count DESC
LIMIT 5;

### **Top 5 Longest Videos**

In [ ]:
INSERT OVERWRITE DIRECTORY '/hdfs/path/Hive/top_5_longest_videos'
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
SELECT video_id, length
FROM merged_youtube_data
ORDER BY length DESC
LIMIT 5;

### **Total Comments Count by Category**

In [ ]:
INSERT OVERWRITE DIRECTORY '/hdfs/path/Hive/total_comments_by_category'
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
SELECT category, SUM(comments_count) AS total_comments
FROM merged_youtube_data
GROUP BY category;

### **Total Number of Videos by Each Uploader**

INSERT OVERWRITE DIRECTORY '/hdfs/path/Hive/total_videos_by_uploader'
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
SELECT uploader, COUNT(video_id) AS video_count
FROM merged_youtube_data
GROUP BY uploader;

### **Average Video Length by Category**

In [ ]:
INSERT OVERWRITE DIRECTORY '/hdfs/path/Hive/avg_video_length_by_category'
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
SELECT category, 
       COALESCE(AVG(length), 0) AS avg_length
FROM merged_youtube_data
WHERE category IS NOT NULL
GROUP BY category;

### **Videos with High Views but Low Ratings**

In [ ]:
INSERT OVERWRITE DIRECTORY '/hdfs/path/Hive/high_views_low_ratings'
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
SELECT video_id, views, rating
FROM merged_youtube_data
WHERE views > 100000 AND rating < 2.5;

### **Relationship Between Video Length and Views**

In [ ]:
INSERT OVERWRITE DIRECTORY '/hdfs/path/Hive/length_vs_views'
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
SELECT length, AVG(views) AS avg_views
FROM merged_youtube_data
GROUP BY length;

# **Pig**

#### Run the following commands on Pig

In [ ]:
youtube_data = LOAD '/hdfs/path/Hive/merged_youtube_data' USING PigStorage('\t') AS (
    video_id: chararray, 
    uploader: chararray, 
    age: float, 
    category: chararray, 
    length: float, 
    views: float, 
    rating: float, 
    ratings_count: float, 
    comments_count: float, 
    related_videos: chararray
);

In [ ]:
DESCRIBE youtube_data;

In [ ]:
limited_data = LIMIT youtube_data 10;
DUMP limited_data;

### **Top 10 Most Viewed Videos**

In [ ]:
top_10_most_viewed_videos = LIMIT (ORDER youtube_data BY views DESC) 10;
STORE top_10_most_viewed_videos INTO '/hdfs/path/Pig/top_10_most_viewed_videos' USING PigStorage(',');

### **Average Rating by Category**

In [ ]:
avg_rating_by_category = FOREACH (GROUP youtube_data BY category) GENERATE 
    group AS category, 
    AVG(youtube_data.rating) AS avg_rating;
STORE avg_rating_by_category INTO '/hdfs/path/Pig/avg_rating_by_category' USING PigStorage(',');

### **Videos with Ratings Greater Than 4.5**

In [ ]:
videos_with_ratings_above_4_5 = FILTER youtube_data BY rating > 4.5;
STORE videos_with_ratings_above_4_5 INTO '/hdfs/path/Pig/videos_with_ratings_above_4_5' USING PigStorage(',');

### **Top 5 Uploaders by Number of Videos**

In [ ]:
grouped_by_uploader = GROUP youtube_data BY uploader;
uploader_video_count = FOREACH grouped_by_uploader GENERATE group AS uploader, COUNT(youtube_data) AS video_count;
sorted_uploader = ORDER uploader_video_count BY video_count DESC;
top_5_uploaders = LIMIT sorted_uploader 5;
STORE top_5_uploaders INTO '/hdfs/path/Pig/top_5_uploaders' USING PigStorage(',');

### **Average Views per Category**

In [ ]:
avg_views_by_category = FOREACH (GROUP youtube_data BY category) GENERATE 
    group AS category, 
    AVG(youtube_data.views) AS avg_views;
STORE avg_views_by_category INTO '/hdfs/path/Pig/avg_views_by_category' USING PigStorage(',');

### **Top 5 Longest Videos**

In [ ]:
top_5_longest_videos = LIMIT (ORDER youtube_data BY length DESC) 5;
STORE top_5_longest_videos INTO '/hdfs/path/Pig/top_5_longest_videos' USING PigStorage(',');

### **Total Comments Count by Category**

In [ ]:
total_comments_by_category = FOREACH (GROUP youtube_data BY category) GENERATE 
    group AS category, 
    SUM(youtube_data.comments_count) AS total_comments;
STORE total_comments_by_category INTO '/hdfs/path/Pig/total_comments_by_category' USING PigStorage(',');

### **Total Number of Videos by Each Uploader**

In [ ]:
total_videos_by_uploader = FOREACH (GROUP youtube_data BY uploader) GENERATE 
    group AS uploader, 
    COUNT(youtube_data.video_id) AS total_videos;
STORE total_videos_by_uploader INTO '/hdfs/path/Pig/total_videos_by_uploader' USING PigStorage(',');

### **Average Video Length by Category**

In [ ]:
avg_video_length_by_category = FOREACH (GROUP youtube_data BY category) GENERATE 
    group AS category, 
    AVG(youtube_data.length) AS avg_length;
STORE avg_video_length_by_category INTO '/hdfs/path/Pig/avg_video_length_by_category' USING PigStorage(',');

### **Videos with High Views but Low Ratings**

In [ ]:
high_views_low_ratings = FILTER youtube_data BY views > 10000 AND rating < 3.0;
STORE high_views_low_ratings INTO '/hdfs/path/Pig/high_views_low_ratings' USING PigStorage(',');

### **Relationship Between Video Length and Views**

In [ ]:
length_vs_views = GROUP youtube_data BY length;
avg_views = FOREACH length_vs_views GENERATE 
    group AS length, 
    AVG(youtube_data.views) AS avg_views;

STORE avg_views INTO '/hdfs/path/Pig/length_vs_views' USING PigStorage('\t');

# **HBase**

#### HBase command before executing the following : create 'youtube_data', 'video_info', 'metrics'

In [ ]:
from pyspark.sql import SparkSession
from happybase import Connection
import happybase

In [ ]:
spark = SparkSession.builder.appName("HiveToHBase").enableHiveSupport().getOrCreate()

In [ ]:
df = spark.read.option("sep", "\t").csv("/hdfs/path/Hive/merged_youtube_data", header=False)

In [ ]:
df.show(3)

In [ ]:
df = df.withColumnRenamed("_c0", "video_id") \
       .withColumnRenamed("_c1", "uploader") \
       .withColumnRenamed("_c2", "age") \
       .withColumnRenamed("_c3", "category") \
       .withColumnRenamed("_c4", "length") \
       .withColumnRenamed("_c5", "views") \
       .withColumnRenamed("_c6", "rating") \
       .withColumnRenamed("_c7", "ratings_count") \
       .withColumnRenamed("_c8", "comments_count") \
       .withColumnRenamed("_c9", "related_videos")

In [ ]:
df.show(3)

In [ ]:
connection = happybase.Connection('localhost')  # Replace 'localhost' with your HBase host
connection.open()
table = connection.table('youtube_data') 

In [ ]:
def write_partition_to_hbase(partition):
    import happybase

    connection = happybase.Connection('localhost')
    table = connection.table('youtube_data')

    try:
        for row in partition:
            row_key = row['video_id'] 
            if row_key:  # Ensure row_key is not None
                data = {
                    b'video_info:uploader': row['uploader'].encode() if row['uploader'] else b'',
                    b'video_info:category': row['category'].encode() if row['category'] else b'',
                    b'video_info:related_videos': row['related_videos'].encode() if row['related_videos'] else b'',
                    b'metrics:length': str(row['length']).encode() if row['length'] is not None else b'',
                    b'metrics:views': str(row['views']).encode() if row['views'] is not None else b'',
                    b'metrics:rating': str(row['rating']).encode() if row['rating'] is not None else b'',
                    b'metrics:ratings_count': str(row['ratings_count']).encode() if row['ratings_count'] is not None else b'',
                    b'metrics:comments_count': str(row['comments_count']).encode() if row['comments_count'] is not None else b''
                }
                table.put(row_key.encode(), data)
    finally:
        connection.close()

In [ ]:
df.repartition(100).foreachPartition(write_partition_to_hbase)

In [ ]:
spark.stop()

In [ ]:
connection = happybase.Connection('localhost')

table = connection.table('youtube_data')

# Scan a few rows to inspect the data
for key, data in table.scan(limit=5): 
    print(key, data)

connection.close()

#### Run the following commands on Hbase to check the successful insertion of the dataset

In [ ]:
list

In [ ]:
count 'youtube_data'

In [ ]:
describe 'youtube_data'

In [ ]:
describe 'youtube_data'   

In [ ]:
scan 'youtube_data', {LIMIT => 2}

#### The output should look somewhat like this 

In [ ]:
ROW                                     COLUMN+CELL                                                                                                        
 ---mkyh9Obc                            column=metrics:comments_count, timestamp=2024-10-25T13:02:17.181, value=1.0                                        
 ---mkyh9Obc                            column=metrics:length, timestamp=2024-10-25T13:02:17.181, value=306.0                                              
 ---mkyh9Obc                            column=metrics:rating, timestamp=2024-10-25T13:02:17.181, value=5.0                                                
 ---mkyh9Obc                            column=metrics:ratings_count, timestamp=2024-10-25T13:02:17.181, value=2.0                                         
 ---mkyh9Obc                            column=metrics:views, timestamp=2024-10-25T13:02:17.181, value=1616.0                                              
 ---mkyh9Obc                            column=video_info:category, timestamp=2024-10-25T13:02:17.181, value=Sports                                        
 ---mkyh9Obc                            column=video_info:related_videos, timestamp=2024-10-25T13:02:17.181, value=nUlccnCi2OY,L_GUfJ_tRLY,S1PxWoU_klE,duMS
                                        cxR7LJ8,ymEdv7jNSSg,UR2CqPMwlzE,QNowKE2FtRE,jBDKKGrYpQw,jxLyjpajW6o,lWbwUhoP_0M,5rkuDqKgAgw,WGbsM1Ud548,pXW-ioHPDac
                                        ,zpcUzWGI-QI,r_CscRAEEg4,xj_mR2r6VL8,Q0C0ndSbiwg,kcBOoYWpB5E,DXvx8HBMTYA,_THwrdK8RbQ                               
 ---mkyh9Obc                            column=video_info:uploader, timestamp=2024-10-25T13:02:17.181, value=A7N7                                          
 ---nH-hN_3E                            column=metrics:comments_count, timestamp=2024-10-25T13:04:33.793, value=3.0                                        
 ---nH-hN_3E                            column=metrics:length, timestamp=2024-10-25T13:04:33.793, value=462.0                                              
 ---nH-hN_3E                            column=metrics:rating, timestamp=2024-10-25T13:04:33.793, value=4.88                                               
 ---nH-hN_3E                            column=metrics:ratings_count, timestamp=2024-10-25T13:04:33.793, value=8.0                                         
 ---nH-hN_3E                            column=metrics:views, timestamp=2024-10-25T13:04:33.793, value=1715.0                                              
 ---nH-hN_3E                            column=video_info:category, timestamp=2024-10-25T13:04:33.793, value=Entertainment                                 
 ---nH-hN_3E                            column=video_info:related_videos, timestamp=2024-10-25T13:04:33.793, value=ES5MMnOot3U,doijMJJQITY,bIPiXgHGeeQ,GVle
                                        FWEbv98,HAFBgb6tVDo,PouuidULyjU,9jNZMg5bXGw,4n8mcj3786M,sIRQJJTy35E,bG5FMcz43EE,NA6fAakyHoM,I3iLmMd4GbI,U24Jag8jMPk
                                        ,mKeKBN1KW1Q,in3PtYmUpKE,cXH90iV6tdc,fhsHowxFudg,EsSRcjiS9Xc,B1fiPXbdVRs,65Cps7kh_co                               
 ---nH-hN_3E                            column=video_info:uploader, timestamp=2024-10-25T13:04:33.793, value=lilxkutie7                                    
2 row(s)
Took 0.1478 seconds                                                                                                                                        

#### Query Execution 

In [ ]:
import happybase
import time
import subprocess

In [ ]:
connection = happybase.Connection('localhost')
connection.open()
table = connection.table('youtube_data')

In [ ]:
# Utility function to measure execution time and save results to HDFS
def measure_execution_time(func, filename, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)  
    end_time = time.time()
    execution_time = end_time - start_time
    
    save_to_hdfs(result, filename)

    print(f"Execution Time for {func.__name__}: {execution_time} seconds")
    return result, execution_time

def save_to_hdfs(data, filename):
    with open(filename, "w") as f:
        for item in data:
            f.write(f"{item}\n")
    
    hdfs_path = f"/hdfs/path/Hbase/{filename}" 
    subprocess.run(["hdfs", "dfs", "-put","-f", filename, hdfs_path], check=True)
    print(f"Results saved to HDFS at {hdfs_path}")

In [ ]:
# Query 1: Top 10 Most Viewed Videos
def top_10_most_viewed_videos():
    all_data = []
    for key, data in table.scan():
        views = data.get(b'metrics:views')
        if views is not None:
            all_data.append((key.decode('utf-8'), int(float(views))))
    top_videos = sorted(all_data, key=lambda x: x[1], reverse=True)[:10]
    return top_videos

In [ ]:
# Query 2: Average Rating by Category
def avg_rating_by_category():
    category_ratings = {}
    for key, data in table.scan():
        category = data.get(b'video_info:category')
        rating = data.get(b'metrics:rating')
        if category and rating:
            category = category.decode('utf-8')
            try:
                rating = float(rating)
            except ValueError:
                continue  # Skip invalid rating values
            if category not in category_ratings:
                category_ratings[category] = {'sum': 0, 'count': 0}
            category_ratings[category]['sum'] += rating
            category_ratings[category]['count'] += 1
    
    avg_ratings = {cat: category_ratings[cat]['sum'] / category_ratings[cat]['count'] for cat in category_ratings}
    return avg_ratings

In [ ]:
# Query 3: Videos with Ratings Greater Than 4.5
def videos_with_high_ratings():
    high_rated_videos = []
    for key, data in table.scan():
        rating = data.get(b'metrics:rating')
        if rating is not None and float(rating) > 4.5:
            high_rated_videos.append((key.decode('utf-8'), float(rating)))
    return high_rated_videos

In [ ]:
# Query 4: Average Views per Category
def avg_views_per_category():
    category_views = {}
    for key, data in table.scan():
        category = data.get(b'video_info:category')
        views = data.get(b'metrics:views')
        if category and views:
            category = category.decode('utf-8')
            try:
                views = float(views)
            except ValueError:
                continue  # Skip invalid view values
            if category not in category_views:
                category_views[category] = {'sum': 0, 'count': 0}
            category_views[category]['sum'] += views
            category_views[category]['count'] += 1
    
    avg_views = {cat: category_views[cat]['sum'] / category_views[cat]['count'] for cat in category_views}
    return avg_views

In [ ]:
# Query 5: Top 5 Uploaders by Number of Videos
def top_5_uploaders_by_videos():
    uploader_count = {}
    for key, data in table.scan():
        uploader = data.get(b'video_info:uploader')
        if uploader:
            uploader = uploader.decode('utf-8')
            if uploader not in uploader_count:
                uploader_count[uploader] = 0
            uploader_count[uploader] += 1
    
    top_uploaders = sorted(uploader_count.items(), key=lambda x: x[1], reverse=True)[:5]
    return top_uploaders

In [ ]:
# Query 6: Top 5 Longest Videos
def top_5_longest_videos():
    all_data = []
    for key, data in table.scan():
        duration = data.get(b'metrics:length')
        if duration is not None:
            all_data.append((key.decode('utf-8'), float(duration)))
    longest_videos = sorted(all_data, key=lambda x: x[1], reverse=True)[:5]
    return longest_videos

In [ ]:
# Query 7: Total Comments Count by Category
def total_comments_by_category():
    category_comments = {}
    for key, data in table.scan():
        category = data.get(b'video_info:category')
        comments = data.get(b'metrics:comments_count')
        if category and comments:
            category = category.decode('utf-8')
            try:
                comments = int(float(comments))
            except ValueError:
                continue  # Skip invalid comment values
            if category not in category_comments:
                category_comments[category] = 0
            category_comments[category] += comments
    
    return category_comments

In [ ]:
# Query 8: Total Number of Videos by Each Uploader
def total_videos_by_uploader():
    uploader_count = {}
    for key, data in table.scan():
        uploader = data.get(b'video_info:uploader')
        if uploader:
            uploader = uploader.decode('utf-8')
            if uploader not in uploader_count:
                uploader_count[uploader] = 0
            uploader_count[uploader] += 1
    
    return uploader_count

In [ ]:
# Query 9: Average Video Length by Category
def avg_video_length_by_category():
    category_length = {}
    for key, data in table.scan():
        category = data.get(b'video_info:category')
        duration = data.get(b'metrics:length')
        if category and duration:
            category = category.decode('utf-8')
            try:
                duration = float(duration)
            except ValueError:
                continue  # Skip invalid length values
            if category not in category_length:
                category_length[category] = {'sum': 0, 'count': 0}
            category_length[category]['sum'] += duration
            category_length[category]['count'] += 1
    
    avg_length = {cat: category_length[cat]['sum'] / category_length[cat]['count'] for cat in category_length}
    return avg_length

In [ ]:
# Query 10: Videos with High Views but Low Ratings
def videos_high_views_low_ratings():
    videos = []
    for key, data in table.scan():
        views = data.get(b'metrics:views')
        rating = data.get(b'metrics:rating')
        if views and rating:
            views = float(views)
            rating = float(rating)
            if views > 100000 and rating < 3.0:
                videos.append((key.decode('utf-8'), views, rating))
    return videos

In [ ]:
# Query 11: Relationship Between Video Length and Views
def video_length_vs_views():
    length_views = {}
    
    for key, data in table.scan():
        duration = data.get(b'metrics:length')
        views = data.get(b'metrics:views')
        if duration and views:
            try:
                duration = float(duration)
                views = float(views)
            except ValueError:
                continue  # Skip invalid values
            if duration not in length_views:
                length_views[duration] = {'sum': 0, 'count': 0}
            length_views[duration]['sum'] += views
            length_views[duration]['count'] += 1
    
    # Calculate the average views for each video length
    avg_length_vs_views = {length: length_views[length]['sum'] / length_views[length]['count'] for length in length_views}
    return avg_length_vs_views

### **Top 10 Most Viwed Videos**

In [ ]:
top_10_most_viewed, exec_time = measure_execution_time(top_10_most_viewed_videos, "top_10_most_viewed")
print("Top 10 Most Viewed Videos:", top_10_most_viewed)

### **Average Rating by Category**

In [ ]:
avg_rating_cat, exec_time = measure_execution_time(avg_rating_by_category, "avg_rating_by_category")
print("Average Rating by Category:", list(avg_rating_cat.items()))

### **Videos with Ratings Greater Than 4.5**

In [ ]:
high_rated_videos, exec_time = measure_execution_time(videos_with_high_ratings, "high_rated_videos")
print("Videos with High Ratings:", high_rated_videos[:5])

### **Average Views per Category**

In [ ]:
avg_views_cat, exec_time = measure_execution_time(avg_views_per_category, "avg_views_per_category")
print("Average Views per Category:", list(avg_views_cat.items()))

### **Top 5 Uploaders by Number of Videos**

In [ ]:
top_uploaders, exec_time = measure_execution_time(top_5_uploaders_by_videos, "top_5_uploaders")
print("Top 5 Uploaders by Number of Videos:", top_uploaders)

### **Top 5 Longest Videos**

In [ ]:
longest_videos, exec_time = measure_execution_time(top_5_longest_videos, "top_5_longest_videos")
print("Top 5 Longest Videos:", longest_videos)

### **Total Comments Count by Category**

In [ ]:
total_comments_cat, exec_time = measure_execution_time(total_comments_by_category, "total_comments_by_category")
print("Total Comments by Category:", list(total_comments_cat.items()))

### **Total Number of Videos by Each Uploader**

In [ ]:
total_videos_uploader, exec_time = measure_execution_time(total_videos_by_uploader, "total_videos_by_uploader")
print("Total Videos by Each Uploader:", list(total_videos_uploader.items())[:5])

### **Average Video Length by Category**

In [ ]:
avg_length_cat, exec_time = measure_execution_time(avg_video_length_by_category, "avg_video_length_by_category")
print("Average Video Length by Category:", list(avg_length_cat.items()))

### **Videos with High Views but Low Ratings**

In [ ]:
videos_high_views_low_ratings, exec_time = measure_execution_time(videos_high_views_low_ratings, "videos_high_views_low_ratings")
print("Videos with High Views but Low Ratings:", videos_high_views_low_ratings[:5])

### **Video Length vs Views**

In [ ]:
length_vs_views, exec_time = measure_execution_time(video_length_vs_views, "length_vs_views")
print("Length vs Views:", list(length_vs_views.items())[:5])

In [ ]:
connection.close()

# **MongoDB**

In [ ]:
import pymongo
from pymongo import MongoClient
from datetime import datetime
import pprint
import pandas as pd
import ast

In [ ]:
print("Pymongo version:", pymongo.__version__)

In [ ]:
csv_file_path = '/path/to/formatted_data.csv'
df = pd.read_csv(csv_file_path)

##### Terminal command : mongoimport --type csv --db YT --collection 2007 --headerline --file /path/to/formatted_data.csv

In [ ]:
client = MongoClient("mongodb://localhost:27017/")

In [ ]:
db = client["YT"]
collection = db["2007"]

In [ ]:
data = df.to_dict(orient="records") 
collection.insert_many(data)

In [ ]:
data = list(collection.find().limit(4))

In [ ]:
df = pd.DataFrame(data)
print(df.head(4))

In [ ]:
# Function to convert 'Related Videos' field from string to list
for doc in collection.find({"Related Videos": {"$type": "string"}}):
    related_videos_str = doc["Related Videos"]
    try:
        # Convert the string to a list
        related_videos_list = ast.literal_eval(related_videos_str)
        
        # Update the document in MongoDB with the list format
        collection.update_one(
            {"_id": doc["_id"]},
            {"$set": {"Related Videos": related_videos_list}}
        )
    except (SyntaxError, ValueError) as e:
        print(f"Skipping document {doc['_id']} due to conversion error: {e}")

print("Conversion complete. 'Related Videos' should now be an array.")

In [ ]:
query_results = []

In [ ]:
def execute_query(query_func):
    start_time = datetime.now()
    result = query_func()
    end_time = datetime.now()
    execution_time = (end_time - start_time).total_seconds()
    
    # Append the query result with execution time to the list, but store only the first 3 records
    query_results.append({
        "query": query_func.__name__,
        "execution_time": execution_time,
        "result": result[:3]  # Only include first 3 records
    })
    
    # Print the execution time and the first 3 records from the result
    print(f"Executed {query_func.__name__} in {execution_time} seconds.")
    pprint.pprint(result[:3])  # Display only the first 3 records
    print("\n" + "-" * 50 + "\n")

In [ ]:
def top_10_most_viewed_videos():
    # Set all fields to 1 to include them in the result
    return list(collection.find(
        {},  # No filter, retrieves all documents
        {
            "Video ID": 1,
            "Uploader": 1,
            "Age": 1,
            "Category": 1,
            "Length": 1,
            "Views": 1,
            "Rating": 1,
            "Ratings Count": 1,
            "Comments Count": 1,
            "Related Videos": 1,
            "_id": 1  # Include the MongoDB ID field
        }
    ).sort("Views", -1).limit(10))

In [ ]:
def average_rating_by_category():
    pipeline = [
        {"$group": {"_id": "$Category", "average_rating": {"$avg": "$Rating"}}}
    ]
    return list(collection.aggregate(pipeline))

In [ ]:
def videos_with_ratings_greater_than_4_5():
    return list(collection.find({"Rating": {"$gt": 4.5}}))

In [ ]:
def average_views_per_category():
    pipeline = [
        {"$group": {"_id": "$Category", "average_views": {"$avg": "$Views"}}}
    ]
    return list(collection.aggregate(pipeline))

In [ ]:
def top_5_uploaders_by_number_of_videos():
    pipeline = [
        {"$group": {"_id": "$Uploader", "video_count": {"$sum": 1}}},
        {"$sort": {"video_count": -1}},
        {"$limit": 5}
    ]
    return list(collection.aggregate(pipeline))

In [ ]:
def top_5_longest_videos():
    return list(collection.find().sort("Length", -1).limit(5))

In [ ]:
def total_comments_count_by_category():
    pipeline = [
        {"$group": {"_id": "$Category", "total_comments": {"$sum": "$Comments Count"}}}
    ]
    return list(collection.aggregate(pipeline))

In [ ]:
def total_number_of_videos_by_each_uploader():
    pipeline = [
        {"$group": {"_id": "$Uploader", "total_videos": {"$sum": 1}}}
    ]
    return list(collection.aggregate(pipeline))

In [ ]:
def average_video_length_by_category():
    pipeline = [
        {"$group": {"_id": "$Category", "average_length": {"$avg": "$Length"}}}
    ]
    return list(collection.aggregate(pipeline))

In [ ]:
def videos_with_high_views_but_low_ratings():
    return list(collection.find(
        {"Views": {"$gt": 100000}, "Rating": {"$lt": 3.0}},
        {
            "Video ID": 1,
            "Uploader": 1,
            "Category": 1,
            "Views": 1,
            "Rating": 1,
            "Length": 1
        }
    ))

In [ ]:
def video_length_vs_views():
    pipeline = [
        {"$match": {"Length": {"$ne": None}, "Views": {"$ne": None}}},  # Filter out documents with missing Length or Views
        {"$group": {
            "_id": "$Length",
            "total_views": {"$sum": "$Views"},
            "view_count": {"$sum": 1}
        }},
        {"$project": {
            "_id": 0,
            "Length": "$_id",
            "average_views": {"$divide": ["$total_views", "$view_count"]}
        }}
    ]
    
    return list(collection.aggregate(pipeline))

In [ ]:
execute_query(top_10_most_viewed_videos)

In [ ]:
execute_query(average_rating_by_category)

In [ ]:
execute_query(videos_with_ratings_greater_than_4_5)

In [ ]:
execute_query(average_views_per_category)

In [ ]:
execute_query(top_5_uploaders_by_number_of_videos)

In [ ]:
execute_query(top_5_longest_videos)

In [ ]:
execute_query(total_comments_count_by_category)

In [ ]:
execute_query(total_number_of_videos_by_each_uploader)

In [ ]:
execute_query(average_video_length_by_category)

In [ ]:
execute_query(videos_with_high_views_but_low_ratings)

In [ ]:
execute_query(video_length_vs_views)

In [ ]:
print("Execution Times:")
for result in query_results:
    print(f"{result['query']}: {result['execution_time']} seconds")

In [ ]:
print(collection.find_one({}, {
    "Video ID": 1,
    "Uploader": 1,
    "Age": 1,
    "Category": 1,
    "Length": 1,
    "Views": 1,
    "Rating": 1,
    "Ratings Count": 1,
    "Comments Count": 1,
    "Related Videos": 1,
    "_id": 1
}))

In [ ]:
client.close()